In [1]:
import pandas as pd
import glob
import numpy as np
import inc.functions as fn
from inc.indicators import apply_all_indicators
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib  # For saving models

symbols = [
    "BTC-USD", "ETH-USD", "SOL-USD", "LTC-USD",
    "ADA-USD", "AVAX-USD", "DOGE-USD", "MATIC-USD",
    "XRP-USD", "PEPE-USD","XLM-USD"
]

In [2]:
for symbol in symbols:
    # 1. Load and prepare
    print(f"Loading data for {symbol}...")
    df = fn.load_crypto_data(symbol)
    df = apply_all_indicators(df)
    df = fn.create_profit_labels(df)

    # 2. Select Features and Target
    df['Crossover_Encoded'] = df['Crossover'].map({'Bullish': 1, 'Bearish': -1}).fillna(0)
    #feature_cols = ['RSI', 'MACD', 'ATR', 'VWAP_1m', 'Volume_SMA', 'Spread']  # You can expand this
    # feature_cols = [
    #     'RSI', 'MACD', 'MACD_Diff', 'MACD_ROC', 'ATR',
    #     '%K', '%D', 'OBV', 'VWAP_15m',
    #     'EMA_5', 'EMA_13', 'EMA_26',
    #     'Volume_SMA', 'Spread',
    #     'Fib_61.8%', 'Fib_100.0%', 'Fib_161.8%','Crossover_Encoded'
    # ]
    feature_cols = [
        'RSI', 'MACD', 'MACD_Diff', 'MACD_ROC', 'Crossover_Encoded',
        'ATR', 'Spread',
        'EMA_5', 'EMA_13', 'EMA_26',
        'Volume', 'Volume_SMA', 'OBV',
        '%K', '%D',
        'VWAP_1m', 'VWAP_15m',
        'Fib_61.8%', 'Fib_100.0%', 'Fib_161.8%', 'Price_Position','Band_Width','Price_vs_Band'
    ]
    
    #X = df[feature_cols]
    X = df[feature_cols].copy() # Use copy to avoid SettingWithCopyWarning
    y = df['Label']

    label_counts = y.value_counts()

    if (label_counts < 2).any():
        print(f"Skipping {symbol} due to insufficient label distribution:\n{label_counts}")
        continue  # Skip model training for this symbol

    # 3. Clean and Normalize features
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.dropna(inplace=True)
    y = y.loc[X.index]  # Keep labels aligned after dropping rows

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 4. Split train/test
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

    # 5. Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 6. Save model and scaler (optional but recommended)
    joblib.dump(model, f"inc/models/{symbol.replace('-', '_')}_rf_model.pkl")
    joblib.dump(scaler, f"inc/models/{symbol.replace('-', '_')}_scaler.pkl")

    print(f"Finished training model for {symbol}")

Loading data for BTC-USD...
Finished training model for BTC-USD
Loading data for ETH-USD...
Finished training model for ETH-USD
Loading data for SOL-USD...
Finished training model for SOL-USD
Loading data for LTC-USD...
Finished training model for LTC-USD
Loading data for ADA-USD...
Finished training model for ADA-USD
Loading data for AVAX-USD...
Finished training model for AVAX-USD
Loading data for DOGE-USD...
Finished training model for DOGE-USD
Loading data for MATIC-USD...
Finished training model for MATIC-USD
Loading data for XRP-USD...
Finished training model for XRP-USD
Loading data for PEPE-USD...
Finished training model for PEPE-USD
Loading data for XLM-USD...
Finished training model for XLM-USD
